<a href="https://colab.research.google.com/github/ClaytonsCreations/AWS_BigData/blob/main/homeImprovementReviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backpor

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-02-21 18:32:39--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.60MB/s    in 0.2s    

2022-02-21 18:32:40 (5.60 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_Improvement_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
home_imp_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Home_Improvement_v1_00.tsv.gz"), sep="\t", header=True)

# Show DataFrame
home_imp_df.show()


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48881148|R215C9BDXTDQOW|B00FR4YQYK|     381800308|SadoTech Model C ...|Home Improvement|          4|            0|          0|   N|                Y|          Four Stars|        good product| 2015-08-31|
|         US|   47882936|R1DTPUV1J57YHA|B00439MYYE|     921341748|iSpring T32M 3.2 ...|Home Improvement|          5|    

In [ ]:
#count rows
home_imp_df.count()

2634781

In [ ]:
home_imp_df = home_imp_df.dropna()

In [ ]:
home_imp_df.count()

2634255

In [ ]:
home_imp_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [ ]:
home_imp_df.select(['review_id', 'customer_id', 'product_id', 'product_parent', 'review_date'])


DataFrame[review_id: string, customer_id: string, product_id: string, product_parent: string, review_date: string]

In [ ]:
from pyspark.sql.types import StringType, DateType, FloatType, IntegerType

home_imp_df = home_imp_df.withColumn("customer_id",home_imp_df["customer_id"].cast(IntegerType())) \
    .withColumn("star_rating",home_imp_df["star_rating"].cast(IntegerType())) \
    .withColumn("product_parent",home_imp_df["product_parent"].cast(IntegerType())) \
    .withColumn("review_date",home_imp_df["review_date"].cast(DateType())) \
    .withColumn("helpful_votes",home_imp_df["helpful_votes"].cast(IntegerType())) \
    .withColumn("total_votes",home_imp_df["total_votes"].cast(IntegerType())) 
home_imp_df.printSchema()



root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



In [ ]:
review_id_table = home_imp_df.select(['review_id', 'customer_id', 'product_id', 'product_parent', 'review_date'])
review_id_table.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R215C9BDXTDQOW|   48881148|B00FR4YQYK|     381800308| 2015-08-31|
|R1DTPUV1J57YHA|   47882936|B00439MYYE|     921341748| 2015-08-31|
| RFAZK5EWKJWOU|   44435471|B00002N762|      56053291| 2015-08-31|
|R2XT8X0O0WS1AL|   28377689|B000QFCP1G|     595928517| 2015-08-31|
|R14GRNANKO2Y2J|   50134766|B00WRCRKOI|     417053744| 2015-08-31|
|R2BLF9VYL24LCQ|   14066511|B00NIH88EW|     275395071| 2015-08-31|
|R1GI9UW5KJ671O|   15211046|B005B9CI96|     856617815| 2015-08-31|
|R2H5CEJN863M86|   14862498|B008L0OMWI|     125102494| 2015-08-31|
| R5PPDHFOZ3SMU|   23617292|B00P9FTC6O|     523110842| 2015-08-31|
| RE1L9IENKJJ7Y|   35820485|B00K6BQEHQ|     797306964| 2015-08-31|
|R3CZ0990QC2Z0H|   47162350|B008BYQCWM|     865874404| 2015-08-31|
|R3UMMD2IO29QSP|   31884789|B003BLHTOU|     183592595| 2015-08

In [ ]:
products = home_imp_df.select(['product_id', 'product_title'])
products.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00FR4YQYK|SadoTech Model C ...|
|B00439MYYE|iSpring T32M 3.2 ...|
|B00002N762|Schlage F10CS V E...|
|B000QFCP1G|Citri-Strip QCG73...|
|B00WRCRKOI|SleekLighting Bul...|
|B00NIH88EW|VDOMUS&reg;Exquis...|
|B005B9CI96|Frigidaire 316075...|
|B008L0OMWI|Anyray® 5-Bulbs 7...|
|B00P9FTC6O|Cambridge 100 pcs...|
|B00K6BQEHQ|EUBUY Silver Tone...|
|B008BYQCWM|Legend 809125 Leg...|
|B003BLHTOU|Forearm Forklift ...|
|B0065I114K|Newer Technology ...|
|B00J9XB6BQ|Rok Hardware 25 P...|
|B00K36NJ0E|Drift Light Using...|
|B000ZIEOHW|Troy DPH11 Profes...|
|B0009YDHKC|Wright Products V...|
|B00LBGVLS2|LiftMaster Garage...|
|B00002N5DY|Heath Zenith SL-5...|
|B000BQOX88|Intermatic T101 2...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
customers = home_imp_df.groupBy('customer_id').agg({'customer_id': 'count'})
customers = customers.withColumnRenamed('count(customer_id)', 'customer_count')
customers.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   28377689|             1|
|   28258386|             2|
|    9967574|             1|
|   25153155|             1|
|   10088068|             1|
|   45657423|             1|
|   19021463|             1|
|   41413793|             2|
|   43789873|             2|
|   12406466|             1|
|   41045019|             1|
|   35535911|             2|
|   47108763|             1|
|   48113150|             1|
|   18201417|             9|
|   26079415|             3|
|   36114891|             7|
|    3712628|             1|
|   37499901|             2|
|   47321438|             2|
+-----------+--------------+
only showing top 20 rows



In [ ]:
vine_table = home_imp_df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine'])
vine_table.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R215C9BDXTDQOW|          4|            0|          0|   N|
|R1DTPUV1J57YHA|          5|            0|          0|   N|
| RFAZK5EWKJWOU|          5|            0|          0|   N|
|R2XT8X0O0WS1AL|          5|            0|          0|   N|
|R14GRNANKO2Y2J|          5|            0|          0|   N|
|R2BLF9VYL24LCQ|          5|            1|          1|   N|
|R1GI9UW5KJ671O|          5|            0|          0|   N|
|R2H5CEJN863M86|          5|            0|          1|   N|
| R5PPDHFOZ3SMU|          5|            0|          0|   N|
| RE1L9IENKJJ7Y|          1|            0|          0|   N|
|R3CZ0990QC2Z0H|          5|            0|          0|   N|
|R3UMMD2IO29QSP|          5|            0|          0|   N|
| R541LE5J3OJH2|          5|            0|          0|   N|
|R10KD9FIE6Y1AS|          5|            

In [ ]:
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>"
config = {"user":"root", "password": <password>, "driver":"org.postgresql.Driver"}

In [ ]:
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)